In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from keras.layers import Dense, BatchNormalization, Dropout, LSTM
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras import callbacks
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score

In [24]:
data = pd.read_csv("Final_Dataset_2023.csv")
data.head(15)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,Predictions
0,70,1,402,0,36,1,216359,1.8,133,1,1,0
1,90,1,124,1,38,0,187570,1.4,139,1,1,0
2,50,0,39,0,39,0,245416,1.0,126,1,1,0
3,75,0,21,0,39,1,280768,0.9,134,1,1,0
4,85,1,96,0,38,0,198585,0.7,139,1,1,1
5,85,1,261,0,58,0,303801,0.7,138,0,0,0
6,62,0,48,1,82,0,18501,0.7,126,0,0,0
7,85,1,93,0,63,0,145685,0.6,137,0,0,0
8,58,0,149,0,34,1,179886,0.6,136,1,1,0
9,70,1,125,0,36,1,259987,0.6,137,1,0,0


In [3]:
#data=data.drop(["time"],axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   age                       100000 non-null  int64  
 1   anaemia                   100000 non-null  int64  
 2   creatinine_phosphokinase  100000 non-null  int64  
 3   diabetes                  100000 non-null  int64  
 4   ejection_fraction         100000 non-null  int64  
 5   high_blood_pressure       100000 non-null  int64  
 6   platelets                 100000 non-null  int64  
 7   serum_creatinine          100000 non-null  float64
 8   serum_sodium              100000 non-null  int64  
 9   sex                       100000 non-null  int64  
 10  smoking                   100000 non-null  int64  
 11  Predictions               100000 non-null  int64  
dtypes: float64(1), int64(11)
memory usage: 9.2 MB


In [4]:
X=data.drop(["Predictions"],axis=1)
y=data["Predictions"]

In [20]:
#Set up a standard scaler for the features
col_names = list(X.columns)
s_scaler = preprocessing.StandardScaler()
X_df= s_scaler.fit_transform(X)
X_df = pd.DataFrame(X_df, columns=col_names)   
X_df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,100000.0,2.512479e-16,1.000005,-1.728267,-0.909723,-0.091178,0.727366,2.773727
anaemia,100000.0,-3.240075e-17,1.000005,-1.065886,-1.065886,0.938187,0.938187,0.938187
creatinine_phosphokinase,100000.0,-1.762146e-17,1.000005,-0.663794,-0.505212,-0.357493,0.022669,10.491217
diabetes,100000.0,-3.609557e-17,1.000005,-0.913413,-0.913413,-0.913413,1.094795,1.094795
ejection_fraction,100000.0,-1.985967e-16,1.000005,-2.546514,-0.700067,-0.138105,0.504138,3.635070
high_blood_pressure,100000.0,2.273737e-17,1.000005,-0.675848,-0.675848,-0.675848,1.479623,1.479623
platelets,100000.0,-3.979039e-17,1.000005,-3.814198,-0.607143,-0.121620,0.429439,11.762333
serum_creatinine,100000.0,1.818989e-17,1.000005,-1.640958,-0.449330,-0.330168,0.027320,12.658570
serum_sodium,100000.0,2.605702e-15,1.000005,-9.746513,-0.399544,0.092402,0.584348,4.027968
sex,100000.0,1.328004e-16,1.000005,-1.404291,-1.404291,0.712103,0.712103,0.712103


In [6]:
#spliting test and training sets
X_train, X_test, y_train,y_test = train_test_split(X_df,y,test_size=0.25,random_state=7)

In [7]:
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=30, # how many epochs to wait before stopping
    restore_best_weights=True)

# Initialising the NN
model = Sequential()

# layers
model.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
model.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 4, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
from tensorflow.keras.optimizers import SGD
# Compiling the ANN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [8]:
# Train the ANN
history = model.fit(X_train, y_train, batch_size = 32, epochs = 500,callbacks=[early_stopping], validation_split=0.2)

Epoch 1/500
1875/1875 [==============================] - 5s 2ms/step - loss: 0.6093 - accuracy: 0.6693 - val_loss: 0.5672 - val_accuracy: 0.7278
Epoch 2/500
1875/1875 [==============================] - 5s 2ms/step - loss: 0.5960 - accuracy: 0.6775 - val_loss: 0.5603 - val_accuracy: 0.7263
Epoch 3/500
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5934 - accuracy: 0.6795 - val_loss: 0.5604 - val_accuracy: 0.7315
Epoch 4/500
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5911 - accuracy: 0.6790 - val_loss: 0.5561 - val_accuracy: 0.7312
Epoch 5/500
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5886 - accuracy: 0.6796 - val_loss: 0.5526 - val_accuracy: 0.7342
Epoch 6/500
1875/1875 [==============================] - 5s 2ms/step - loss: 0.5853 - accuracy: 0.6817 - val_loss: 0.5460 - val_accuracy: 0.7365
Epoch 7/500
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5878 - accuracy: 0.6806 - val_loss: 0.5507 - val_ac

In [9]:
y_pred= (model.predict(X_test)>0.5).astype(int).flatten()
y_pred

782/782 [==============================] - 1s 1ms/step


array([0, 1, 1, ..., 0, 0, 0])

In [10]:
y_pred= y_pred.astype(bool)
y_pred

array([False,  True,  True, ..., False, False, False])

In [11]:
cm=confusion_matrix(y_test, y_pred)
print(cm)
print(f1_score(y_test,y_pred))

[[13960  1670]
 [ 4857  4513]]
0.5803381984183116


In [12]:
val_accuracy = np.mean(history.history['val_accuracy'])
print("\n%s: %.2f%%" % ('val_accuracy', val_accuracy*100))


val_accuracy: 73.63%


In [13]:
print(accuracy_score(y_test,y_pred))

0.73892


In [40]:
new_data1 = np.array([[85,1,96,0,38,0,198585,0.7,139,1,1]])
new_data2 = np.array([[57,1,121,0,32,0,292119,2.3,147,0,0]])
new_data3 = np.array([[70,1,402,0,36,1,216359,1.8,133,1,1]])
result= s_scaler.transform(new_data3)
y_pred= (model.predict(result)>0.5).astype(int).flatten()

1/1 [==============================] - 0s 22ms/step


D:\SOFTWARE\Anaconda\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [41]:
y_pred

array([1])

In [43]:
new_data2

array([[5.70000e+01, 1.00000e+00, 1.21000e+02, 0.00000e+00, 3.20000e+01,
        0.00000e+00, 2.92119e+05, 2.30000e+00, 1.47000e+02, 0.00000e+00,
        0.00000e+00]])